In [1]:
import torch
import nltk
import json
import torch

nltk.download('stopwords')
nltk_stopwords = set(nltk.corpus.stopwords.words('english'))

colleges_list = open('all-colleges.txt').readlines()
colleges_list = [college[:college.index(' (')] for college in colleges_list]

try:
    data = json.load(open('standardized_output.json', 'r'))
except:
    data = json.load(open('output_2.json'))
    for post in data.values():
        drop = []
        for i in range(len(post['results'])):
            college = post['results'][i]
            closest_name = difflib.get_close_matches(college['school_name'], colleges_list, n=1, cutoff=0.8)
            if closest_name:
                college['school_name'] = closest_name[0]
            else:
                drop.append(i)
        for index in reversed(drop):
            post['results'].pop(index)

    json.dump(data, open('standardized_output.json', 'w'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Yourui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
device

device(type='mps')

In [3]:
from regressor import ResultRegressor, CollegeResultsDataset

try:
    dataset = torch.load('dataset.pt')
except:
    dataset = CollegeResultsDataset(data, colleges_list, nltk_stopwords)
    torch.save(dataset, 'dataset.pt')

full_data_size = len(dataset)
train_size = int(full_data_size * 0.8)
print(f"Train Data Size: {train_size}")

batch_size = 10
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, full_data_size - train_size])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

/Users/Yourui/Documents/nochances/nochances/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/Yourui/Documents/nochances/categorization/regressor.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.model(**inputs)['last_hidden_state'][0][0], dtype=torch.float32)


0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
Loaded rtpx67
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
Loaded rum7il
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
Loaded rutvbd
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3 5
4 100
5 5
6 5
7 2
8 4
9 1789
0 2
1 3
2 5
3

In [6]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, n_epochs, device):
    for epoch in range(n_epochs):
        for param_group in optimizer.param_groups:
            print(f"Current learning rate: {param_group['lr']}")

        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for i, batch in enumerate(train_loader):
            optimizer.zero_grad()

            for k, v in batch.items():
                batch[k] = v.to(device)

            outputs = model(batch)
            loss = criterion(outputs, batch['target'])

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item()
            train_total += batch['target'].size(0)
            train_correct += ((outputs > 0.5) == batch['target']).sum().item()

        train_loss /= len(train_loader)
        train_acc = train_correct / train_total

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch in val_loader:
                for k, v in batch.items():
                    batch[k] = v.to(device)

                outputs = model(batch)
                loss = criterion(outputs, batch['target'])

                val_loss += loss.item()
                val_total += batch['target'].size(0)
                val_correct += ((outputs > 0.5) == batch['target']).sum().item()

                print(outputs)

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total
        scheduler.step()

        print(f'Epoch {epoch+1}/{n_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        torch.save(model.state_dict(), '2023_24_regression.pt')

    return model

In [7]:
model = ResultRegressor().to(device)

criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.00001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000000, gamma=1.0)

n_epochs = 50
model = train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, n_epochs, device)

Current learning rate: 1e-05


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/Yourui/Documents/nochances/nochances/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/Yourui/Documents/nochances/nochances/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Yourui/Documents/nochances/nochances/lib/python3.12/site-packages/torch/utils/data/dataset.py", line 419, in __getitems__
    return [self.dataset[self.indices[idx]] for idx in indices]
            ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
KeyError: 2636


In [ ]:
torch.save(model.state_dict(), '2023_24_regression.pt')